In [5]:
import pandas as pd
import altair as alt

alt.renderers.enable("colab")

RendererRegistry.enable('colab')

In [6]:
df = pd.read_csv("/content/drive/MyDrive/csvs/pivote.csv")
df.head()

,Ano,Conflicto,Decepcion personal,Dolor fisico,Dolor psicologico,Dolor psicologico y dolor fisico,Dolor psicologico/Dolor fisico,Evento positivo,Perdida,Perdida y dolor psicologico,Sufrimiento ajeno,Sufrimiento psicologico
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,1988.0,NaN,NaN,NaN,70.0,NaN,NaN,22.0,NaN,NaN,NaN,NaN
3,1989.0,NaN,NaN,NaN,7.0,NaN,NaN,15.0,3.0,NaN,49.0,NaN
4,1992.0,NaN,NaN,3.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN


In [7]:
year_col = df.columns[0]  # first column = year

df_long = df.melt(
    id_vars=year_col,
    var_name="categoria",
    value_name="duracion"
)

df_long = df_long.dropna(subset=["duracion"])

# Unify 'Sufrimiento psicologico' and 'Dolor psicologico'
df_long['categoria'] = df_long['categoria'].replace('Sufrimiento psicologico', 'Dolor psicologico')

df_long

,Ano,categoria,duracion
8,2008.0,Conflicto,12.0
21,2023.0,Decepcion personal,3.0
26,1992.0,Dolor fisico,3.0
34,1986.0,Dolor psicologico,1.0
35,1988.0,Dolor psicologico,70.0
36,1989.0,Dolor psicologico,7.0
39,2001.0,Dolor psicologico,81.0
41,2008.0,Dolor psicologico,80.0
42,2013.0,Dolor psicologico,16.0
43,2023.0,Dolor psicologico,10.0


In [8]:
custom_colors = [
    "#FFF7E6",  # warm paper cream
    "#F3E6C8",  # light wheat
    "#E8DDB5",  # sunlit parchment
    "#DCECC9",  # pale meadow green
    "#C8E0CC",  # mint-sage
    "#BFD7EA",  # soft sky blue
    "#D7C9E3",  # pastel lavender
    "#F4C6B8",  # peach blossom
    "#E8B4A6",  # soft terracotta
    "#B7D3A8",  # fresh moss
    "#9FC6B8"   # calm teal-sage
]

# Get the unique categories to map them to the custom colors
unique_categories = sorted(df_long['categoria'].unique())

# Adjust custom_colors if necessary after merging categories
# If the number of unique categories is less than the custom_colors list,
# we will slice the custom_colors list to match.
if len(unique_categories) < len(custom_colors):
    palette_colors = custom_colors[:len(unique_categories)]
else:
    palette_colors = custom_colors

palette = alt.Scale(domain=unique_categories, range=palette_colors)

In [9]:
area = alt.Chart(df_long).mark_area(
    interpolate="monotone"
).encode(
    x=alt.X(f"{year_col}:O", title="Año"),
    y=alt.Y(
        "sum(duracion):Q",
        stack="zero",
        title="Duración total"
    ),
    color=alt.Color(
        "categoria:N",
        scale=palette, # Use the custom palette
        legend=alt.Legend(
            orient="bottom",
            direction="horizontal",
            columns=4,
            title=None
        )
    ),
    tooltip=[
        alt.Tooltip("categoria:N", title="Categoría"),
        alt.Tooltip(
            "sum(duracion):Q",
            title="Duración"
        )
    ]
).properties(
    width=800,
    height=280
)

total_line = alt.Chart(df_long).mark_line(
    color="#444",
    strokeDash=[5, 4],
    strokeWidth=2
).encode(
    x=f"{year_col}:O",
    y="sum(duracion):Q"
)

chart = (area + total_line)

In [10]:
chart = chart.configure_view(
    stroke=None
).configure_axis(
    grid=False,
    labelColor="#333",
    titleColor="#333",
    labelFont="sans-serif",
    titleFont="sans-serif"
).configure_legend(
    labelFont="sans-serif",
    labelFontSize=11,
    symbolSize=120
).properties(
    background="#eef6ff"
)

chart

alt.LayerChart(...)

In [11]:
num_unique_categories = df_long['categoria'].nunique()
print(f"Number of unique categories: {num_unique_categories}")

Number of unique categories: 10


In [12]:
chart.save("ghibli_chart.html")

In [13]:
from google.colab import files
files.download("ghibli_chart.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>